<a href="https://colab.research.google.com/github/mr7495/COVID-CT-Code/blob/master/Automated_covid_detector_without_selection_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Automatic COVID detection from patient's CT scan folder
# In this code the CT selection algorithm has not been used

In [1]:
# idea from https://github.com/mr7495/COVID-CT-Code
# developer email : mr7495@yahoo.com

In [ ]:
!nvidia-smi #show the allocated GPU

In [ ]:
#Connect your Google Drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#upgrade and install essential libraries
#Install essential libraries
!pip install git+https://github.com/mr7495/RetinaNet

In [1]:
import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

In [2]:
shape=(512,512,1) #shape of the dataset images (in TIFF format)

In [ ]:
#The fully automated patient classification network
Path='drive/My Drive/COVID-CTset/trained_networks/FPN-fold1.hdf5' # You can use this trained model : 'drive/My Drive/COVID-CTset/trained_networks/FPN-fold1.hdf5'
data_path= CT_folder  #path to the folder containing patient's CT scan images

selected_imgs_path=[]
for r,d,f in os.walk(data_path): 
  for file in f:
    if '.tif' in file:
      selected_imgs_path.append(os.path.join(r,file)) #get the images path from the data_folder

k.clear_session() #clear keras backend
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} #Upsampling layer that was used in feature pyramid network
net=keras.models.load_model(Path, custom_objects=custom_object)


covid_label=0 # index of COVID-19 class 
normal_label=1  # index of normal class 
normal_predicted=0
covid_predicted=0
for img_name in selected_imgs_path: #read images
  img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
  pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #predicted label
  if pred_ind==normal_label:
    normal_predicted+=1
  else:
    covid_predicted+=1

all_imgs_num=len(selected_imgs_path) 
if covid_predicted>=0.1*all_imgs_num: #if at least 10% of a patient CT scans be classified as covid
  print('The patient is identified as infected to COVID-19')
else:
  print('The patient is identified as normal')